# Binary Tree (with optional Binary Search Tree behavior)

A binary tree is a hierarchical data structure where each node has at most two children: `left` and `right`.

This notebook provides:
- A generic `BinaryTree` struct with fundamental operations and traversals.
- An optional BST mode (`as_bst=true`) where inserts/searches enforce the Binary Search Tree ordering.
- Simple, well-typed Rust implementation, plus sample usage for both generic and BST modes.

Notes:
- Duplicates in BST mode go to the right subtree by convention here.
- You can pass a comparison function to define the ordering for complex objects.
- In generic mode, you can manually attach children or use level-order `insert()` to fill the next available slot.


In [2]:
use std::collections::VecDeque;
use std::cmp::Ordering;

// Node represents a binary tree node
#[derive(Debug, Clone)]
pub struct Node<T> {
    pub value: T,
    pub left: Option<Box<Node<T>>>,
    pub right: Option<Box<Node<T>>>,
}

impl<T> Node<T> {
    pub fn new(value: T) -> Self {
        Node {
            value,
            left: None,
            right: None,
        }
    }
}

// BinaryTree represents a binary tree with optional BST behavior
pub struct BinaryTree<T> {
    root: Option<Box<Node<T>>>,
    as_bst: bool,
}

impl<T> BinaryTree<T>
where
    T: Clone + PartialEq + PartialOrd,
{
    // Create a new binary tree
    pub fn new(as_bst: bool) -> Self {
        BinaryTree {
            root: None,
            as_bst,
        }
    }

    // Insert a value into the tree
    pub fn insert(&mut self, value: T) {
        if self.root.is_none() {
            self.root = Some(Box::new(Node::new(value)));
            return;
        }

        if self.as_bst {
            Self::insert_bst(&mut self.root, value);
        } else {
            self.insert_level_order(value);
        }
    }

    // Insert in level-order (breadth-first)
    fn insert_level_order(&mut self, value: T) {
        let mut queue = VecDeque::new();
        if let Some(root) = &self.root {
            queue.push_back(root.as_ref() as *const Node<T> as *mut Node<T>);
        }

        while let Some(curr_ptr) = queue.pop_front() {
            let curr = unsafe { &mut *curr_ptr };
            
            if curr.left.is_none() {
                curr.left = Some(Box::new(Node::new(value)));
                return;
            } else {
                queue.push_back(curr.left.as_ref().unwrap().as_ref() as *const Node<T> as *mut Node<T>);
            }

            if curr.right.is_none() {
                curr.right = Some(Box::new(Node::new(value)));
                return;
            } else {
                queue.push_back(curr.right.as_ref().unwrap().as_ref() as *const Node<T> as *mut Node<T>);
            }
        }
    }

    // Insert maintaining BST ordering
    fn insert_bst(node: &mut Option<Box<Node<T>>>, value: T) {
        if let Some(n) = node {
            if value < n.value {
                if n.left.is_none() {
                    n.left = Some(Box::new(Node::new(value)));
                } else {
                    Self::insert_bst(&mut n.left, value);
                }
            } else {
                // duplicates go right
                if n.right.is_none() {
                    n.right = Some(Box::new(Node::new(value)));
                } else {
                    Self::insert_bst(&mut n.right, value);
                }
            }
        }
    }

    // Search for a value in the tree
    pub fn search(&self, value: &T) -> bool {
        if self.as_bst {
            Self::search_bst(&self.root, value)
        } else {
            self.search_linear(value)
        }
    }

    // Search using BST ordering
    fn search_bst(node: &Option<Box<Node<T>>>, value: &T) -> bool {
        match node {
            None => false,
            Some(n) => {
                match value.partial_cmp(&n.value) {
                    Some(Ordering::Equal) => true,
                    Some(Ordering::Less) => Self::search_bst(&n.left, value),
                    _ => Self::search_bst(&n.right, value),
                }
            }
        }
    }

    // Search linearly through all nodes
    fn search_linear(&self, value: &T) -> bool {
        for node_value in self.level_order() {
            if node_value == *value {
                return true;
            }
        }
        false
    }

    // Check if a value exists in the tree
    pub fn contains(&self, value: &T) -> bool {
        self.search(value)
    }

    // Preorder traversal: Node -> Left -> Right
    pub fn preorder(&self) -> Vec<T> {
        let mut result = Vec::new();
        Self::preorder_helper(&self.root, &mut result);
        result
    }

    fn preorder_helper(node: &Option<Box<Node<T>>>, result: &mut Vec<T>) {
        if let Some(n) = node {
            result.push(n.value.clone());
            Self::preorder_helper(&n.left, result);
            Self::preorder_helper(&n.right, result);
        }
    }

    // Inorder traversal: Left -> Node -> Right
    pub fn inorder(&self) -> Vec<T> {
        let mut result = Vec::new();
        Self::inorder_helper(&self.root, &mut result);
        result
    }

    fn inorder_helper(node: &Option<Box<Node<T>>>, result: &mut Vec<T>) {
        if let Some(n) = node {
            Self::inorder_helper(&n.left, result);
            result.push(n.value.clone());
            Self::inorder_helper(&n.right, result);
        }
    }

    // Postorder traversal: Left -> Right -> Node
    pub fn postorder(&self) -> Vec<T> {
        let mut result = Vec::new();
        Self::postorder_helper(&self.root, &mut result);
        result
    }

    fn postorder_helper(node: &Option<Box<Node<T>>>, result: &mut Vec<T>) {
        if let Some(n) = node {
            Self::postorder_helper(&n.left, result);
            Self::postorder_helper(&n.right, result);
            result.push(n.value.clone());
        }
    }

    // Level-order traversal
    pub fn level_order(&self) -> Vec<T> {
        let mut result = Vec::new();
        if self.root.is_none() {
            return result;
        }

        let mut queue = VecDeque::new();
        queue.push_back(self.root.as_ref().unwrap().as_ref());

        while let Some(curr) = queue.pop_front() {
            result.push(curr.value.clone());

            if let Some(left) = &curr.left {
                queue.push_back(left.as_ref());
            }
            if let Some(right) = &curr.right {
                queue.push_back(right.as_ref());
            }
        }
        result
    }

    // Get the height of the tree
    pub fn height(&self) -> i32 {
        Self::height_helper(&self.root)
    }

    fn height_helper(node: &Option<Box<Node<T>>>) -> i32 {
        match node {
            None => -1, // height of empty tree
            Some(n) => {
                let left_height = Self::height_helper(&n.left);
                let right_height = Self::height_helper(&n.right);
                1 + left_height.max(right_height)
            }
        }
    }

    // Check if the tree is empty
    pub fn is_empty(&self) -> bool {
        self.root.is_none()
    }

    // Clear all nodes from the tree
    pub fn clear(&mut self) {
        self.root = None;
    }
}

## Examples

Below are quick examples showing generic mode (level-order inserts) and BST mode (ordered inserts and search).


In [3]:
// Generic mode: level-order insertion
let mut bt = BinaryTree::new(false);
let values = vec![1, 2, 3, 4, 5, 6];
for v in values {
    bt.insert(v);
}

/*
tree
         1
        / \
       2   3
     / \  /
    4  5 6
*/

println!("Generic tree level-order: {:?}", bt.level_order());
println!("Preorder: {:?}", bt.preorder());
println!("Inorder: {:?}", bt.inorder());
println!("Postorder: {:?}", bt.postorder());
println!("Height: {}", bt.height());
println!("Contains 5? {}", bt.contains(&5));
println!("Contains 42? {}", bt.contains(&42));

Generic tree level-order: [1, 2, 3, 4, 5, 6]
Preorder: [1, 2, 4, 5, 3, 6]
Inorder: [4, 2, 5, 1, 6, 3]
Postorder: [4, 5, 2, 6, 3, 1]
Height: 2
Contains 5? true
Contains 42? false
Preorder: [1, 2, 4, 5, 3, 6]
Inorder: [4, 2, 5, 1, 6, 3]
Postorder: [4, 5, 2, 6, 3, 1]
Height: 2
Contains 5? true
Contains 42? false


In [4]:
// BST mode: ordered insert/search
let mut bst = BinaryTree::new(true);
let bst_values = vec![8, 3, 10, 1, 6, 14, 4, 7, 13];
for v in bst_values {
    bst.insert(v);
}

println!("BST inorder (sorted): {:?}", bst.inorder());
println!("BST level-order: {:?}", bst.level_order());
println!("Search 7: {}", bst.search(&7));
println!("Search 2: {}", bst.search(&2));
println!("Height: {}", bst.height());

BST inorder (sorted): [1, 3, 4, 6, 7, 8, 10, 13, 14]
BST level-order: [8, 3, 10, 1, 6, 14, 4, 7, 13]
Search 7: true
Search 2: false
Height: 3
BST level-order: [8, 3, 10, 1, 6, 14, 4, 7, 13]
Search 7: true
Search 2: false
Height: 3


In [5]:
// Using custom struct with ordering
#[derive(Debug, Clone, PartialEq, PartialOrd)]
struct Person {
    name: String,
    age: u32,
}

impl Person {
    fn new(name: &str, age: u32) -> Self {
        Person {
            name: name.to_string(),
            age,
        }
    }
}

let mut people_bst = BinaryTree::new(true);
let people = vec![
    Person::new("Ann", 30),
    Person::new("Ben", 25),
    Person::new("Cara", 40),
];

for p in people {
    people_bst.insert(p);
}

println!("People inorder (sorted by age): {:?}", people_bst.inorder());
let search_person = Person::new("X", 25);
println!("Search by age=25: {}", people_bst.search(&search_person));

People inorder (sorted by age): [Person { name: "Ann", age: 30 }, Person { name: "Ben", age: 25 }, Person { name: "Cara", age: 40 }]
Search by age=25: false
Search by age=25: false
